#### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgbm
import ipywidgets
import statistics
import gc
import os
import sys
import dd as dd
import optuna
import dill

from catboost import Pool, CatBoostClassifier, CatBoostRanker
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import KFold
from collections import defaultdict
from copy import deepcopy
from itertools import product
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from optuna.integration import LightGBMPruningCallback

sys.path.append('../Credit Scoring/')

In [2]:
path = 'data/train_data/'

In [3]:
def make_dataset(list_scores):
    
    list_scores_model = [list_scores]
    print(list_scores_model)
    data = {
        'Модель': [item[0] for item in list_scores_model],
        'Fold 1': [item[1][0] for item in list_scores_model],
        'Fold 2': [item[1][1] for item in list_scores_model],
        'Fold 3': [item[1][2] for item in list_scores_model],
        'Fold 4': [item[1][3] for item in list_scores_model],
        'Fold 5': [item[1][4] for item in list_scores_model],
        'Mean roc_auc_score': [statistics.mean(list_scores_model[0][1])]
    }
    
    return pd.DataFrame(data)

def make_df_for_modelling(data, target: str):
    
    data[['id', 'flag']] = data[['id', 'flag']].astype(int)
    data.drop(['id'], axis=1, inplace=True)

    features = list(data.columns.values)
    features.remove(target)

    X = data[features]
    y = data[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify = y, random_state=42)
    
    return X_train, X_test, y_train, y_test

In [4]:
def pipeline():

    data = dd.modify_data()
    X_train, X_test, y_train, y_test = make_df_for_modelling(data, 'flag')
    
    return X_train, X_test, y_train, y_test

In [5]:
gc.collect()

0

#### Загрузка данных

In [6]:
X_train, X_test, y_train, y_test = pipeline()

Modify data!


Transforming transactions data:   0%|          | 0/12 [00:00<?, ?it/s]

['data/train_data\\train_data_0.pq', 'data/train_data\\train_data_1.pq', 'data/train_data\\train_data_10.pq', 'data/train_data\\train_data_11.pq', 'data/train_data\\train_data_2.pq', 'data/train_data\\train_data_3.pq', 'data/train_data\\train_data_4.pq', 'data/train_data\\train_data_5.pq', 'data/train_data\\train_data_6.pq', 'data/train_data\\train_data_7.pq', 'data/train_data\\train_data_8.pq', 'data/train_data\\train_data_9.pq']
Reading chunks!
Read chunk: data/train_data\train_data_0.pq


Reading chunks with pandas!:   0%|          | 0/1 [00:00<?, ?it/s]

Chunk data/train_data\train_data_0.pq
['data/train_data\\train_data_0.pq', 'data/train_data\\train_data_1.pq', 'data/train_data\\train_data_10.pq', 'data/train_data\\train_data_11.pq', 'data/train_data\\train_data_2.pq', 'data/train_data\\train_data_3.pq', 'data/train_data\\train_data_4.pq', 'data/train_data\\train_data_5.pq', 'data/train_data\\train_data_6.pq', 'data/train_data\\train_data_7.pq', 'data/train_data\\train_data_8.pq', 'data/train_data\\train_data_9.pq']
Reading chunks!
Read chunk: data/train_data\train_data_1.pq


...

Reading chunks with pandas!:   0%|          | 0/1 [00:00<?, ?it/s]

Chunk data/train_data\train_data_9.pq
['data/preprocess_train_data\\processed_chunk_count_agg_000.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_001.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_002.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_003.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_004.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_005.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_006.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_007.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_008.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_009.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_010.parquet', 'data/preprocess_train_data\\processed_chunk_count_agg_011.parquet']
Reading chunks!


Reading chunks in COUNT frames:   0%|          | 0/12 [00:00<?, ?it/s]

Reading chunks in WEIGHT frames:   0%|          | 0/12 [00:00<?, ?it/s]

Фрейм сохранен по пути data/process_data\train_data_w_target.csv


In [7]:
print('Размеры обучающей выборки:', X_train.shape)
print('Размеры тестовой выбоки', X_test.shape)

Размеры обучающей выборки: (2400000, 839)
Размеры тестовой выбоки (600000, 839)


In [8]:
agg_model_scores = []

In [9]:
def make_df_for_modelling(path_to_data: str, target: str):
    
    data = pd.read_csv(os.path.join(path, 'train_data_w_target.csv'), index_col=0)
    data[['id', 'flag']] = data[['id', 'flag']].astype(int)
    data.drop(['id'], axis=1, inplace=True)

    features = list(data.columns.values)
    features.remove(target)

    X = data[features]
    y = data[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify = y, random_state=42)
    
    return X_train, X_test, y_train, y_test

#### Catboost Classifier

In [10]:
catboost = CatBoostClassifier(
            eval_metric='AUC',
            verbose=1000,
            task_type='GPU',
            iterations=5000,
            early_stopping_rounds=400,
            loss_function='Logloss',
            learning_rate=0.01,
            auto_class_weights='Balanced',
            random_state=42,
            boosting_type='Ordered',
            bootstrap_type='Bernoulli',
            subsample=0.5,
            leaf_estimation_iterations=5,
            max_ctr_complexity=1)

params = {
    'depth': [4, 5, 6],
    'l2_leaf_reg': [2, 3, 4]   
}

search_results = catboost.grid_search(params, X_train, y_train, cv=5, plot=True, verbose=1000)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6659016	best: 0.6659016 (0)	total: 361ms	remaining: 30m 3s
1000:	test: 0.7508651	best: 0.7508651 (1000)	total: 2m 29s	remaining: 9m 56s
2000:	test: 0.7537392	best: 0.7537419 (1939)	total: 4m 27s	remaining: 6m 41s
3000:	test: 0.7542410	best: 0.7542414 (2981)	total: 6m 13s	remaining: 4m 9s
4000:	test: 0.7545777	best: 0.7545777 (4000)	total: 7m 59s	remaining: 1m 59s
4999:	test: 0.7559461	best: 0.7559461 (4996)	total: 9m 54s	remaining: 0us
bestTest = 0.75594607
bestIteration = 4996
Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
0:	loss: 0.7559461	best: 0.7559461 (0)	total: 10m	remaining: 1h 20m 7s


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6659016	best: 0.6659016 (0)	total: 170ms	remaining: 14m 9s
1000:	test: 0.7513872	best: 0.7513872 (1000)	total: 2m 30s	remaining: 10m 1s
2000:	test: 0.7564647	best: 0.7564647 (2000)	total: 4m 41s	remaining: 7m 2s
3000:	test: 0.7578177	best: 0.7578179 (2960)	total: 6m 37s	remaining: 4m 25s
4000:	test: 0.7580073	best: 0.7580073 (3975)	total: 8m 22s	remaining: 2m 5s
4999:	test: 0.7582775	best: 0.7582775 (4997)	total: 10m 7s	remaining: 0us
bestTest = 0.7582775354
bestIteration = 4997
Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.


...

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6897177	best: 0.6897177 (0)	total: 268ms	remaining: 22m 17s
1000:	test: 0.7563851	best: 0.7563851 (1000)	total: 3m 51s	remaining: 15m 26s
2000:	test: 0.7607524	best: 0.7607524 (1999)	total: 7m 5s	remaining: 10m 37s
3000:	test: 0.7626578	best: 0.7626578 (3000)	total: 10m 6s	remaining: 6m 44s
4000:	test: 0.7636035	best: 0.7636038 (3993)	total: 13m 7s	remaining: 3m 16s
4999:	test: 0.7641729	best: 0.7641729 (4999)	total: 15m 59s	remaining: 0us
bestTest = 0.764172852
bestIteration = 4999
Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
8:	loss: 0.7641729	best: 0.7642974 (5)	total: 1h 38m 54s	remaining: 0us
Estimating final quality...


Default metric period is 5 because AUC is/are not implemented for GPU


Training on fold [0/5]
0:	test: 0.6807334	best: 0.6807334 (0)	total: 224ms	remaining: 18m 39s
1000:	test: 0.7546948	best: 0.7546948 (999)	total: 3m 1s	remaining: 12m 3s
2000:	test: 0.7596336	best: 0.7596336 (1998)	total: 5m 40s	remaining: 8m 29s
3000:	test: 0.7621096	best: 0.7621096 (2999)	total: 8m 10s	remaining: 5m 26s
4000:	test: 0.7630756	best: 0.7630756 (3982)	total: 10m 18s	remaining: 2m 34s
4999:	test: 0.7634505	best: 0.7634505 (4999)	total: 12m 12s	remaining: 0us
bestTest = 0.7634504735
bestIteration = 4999
Training on fold [1/5]
0:	test: 0.6753471	best: 0.6753471 (0)	total: 367ms	remaining: 30m 34s
1000:	test: 0.7528476	best: 0.7528476 (1000)	total: 3m 3s	remaining: 12m 12s
2000:	test: 0.7573181	best: 0.7573181 (2000)	total: 5m 34s	remaining: 8m 21s
3000:	test: 0.7601995	best: 0.7601995 (3000)	total: 8m 11s	remaining: 5m 27s
4000:	test: 0.7619949	best: 0.7619949 (4000)	total: 10m 51s	remaining: 2m 42s
4999:	test: 0.7633466	best: 0.7633477 (4997)	total: 13m 39s	remaining: 0us
b

Default metric period is 5 because AUC is/are not implemented for GPU


In [11]:
best_params_catboost = catboost.get_params()

In [12]:
gc.collect()

0

In [13]:
cv = StratifiedKFold(n_splits = 5, random_state=42, shuffle=True)

models_catboost = []
metrics_catboost = []

for i, (train_indexes, test_indexes) in enumerate(cv.split(X_train, y_train), start=1):
    
    print(f'Fold {i} started')
    
    model = CatBoostClassifier(**best_params_catboost)
    
    y_tr, y_v = y_train.iloc[train_indexes], y_train.iloc[test_indexes]
    X_tr, X_v = X_train.iloc[train_indexes], X_train.iloc[test_indexes]
    
    fit_model = model.fit(X_tr, y_tr, eval_set=(X_v.values, y_v.values),
                          use_best_model = True, plot=False)
    
    predictions = fit_model.predict_proba(X_v)[:, 1]
    print('ROC AUC SCORE:', roc_auc_score(y_v, predictions))
    
    models_catboost.append(fit_model)
    metrics_catboost.append(roc_auc_score(y_v, predictions))


Fold 1 started


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6791353	best: 0.6791353 (0)	total: 225ms	remaining: 18m 46s
1000:	test: 0.7566491	best: 0.7566491 (1000)	total: 3m 8s	remaining: 12m 31s
2000:	test: 0.7625438	best: 0.7625438 (2000)	total: 6m 5s	remaining: 9m 8s
3000:	test: 0.7647199	best: 0.7647201 (2999)	total: 8m 50s	remaining: 5m 53s
4000:	test: 0.7660391	best: 0.7660391 (4000)	total: 11m 38s	remaining: 2m 54s
4999:	test: 0.7670122	best: 0.7670122 (4999)	total: 14m 24s	remaining: 0us
bestTest = 0.7670122087
bestIteration = 4999
ROC AUC SCORE: 0.7670122242726762
Fold 2 started


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6766360	best: 0.6766360 (0)	total: 229ms	remaining: 19m 3s
1000:	test: 0.7541778	best: 0.7541778 (999)	total: 3m 3s	remaining: 12m 13s
2000:	test: 0.7596983	best: 0.7596983 (2000)	total: 5m 43s	remaining: 8m 34s
3000:	test: 0.7630168	best: 0.7630168 (3000)	total: 8m 41s	remaining: 5m 47s
4000:	test: 0.7642345	best: 0.7642345 (4000)	total: 11m 26s	remaining: 2m 51s
4999:	test: 0.7643392	best: 0.7643397 (4997)	total: 13m 45s	remaining: 0us
bestTest = 0.7643396854
bestIteration = 4997
Shrink model to first 4998 iterations.
ROC AUC SCORE: 0.7643396464970058
Fold 3 started


...

0:	test: 0.6770013	best: 0.6770013 (0)	total: 222ms	remaining: 18m 31s
1000:	test: 0.7534680	best: 0.7534680 (1000)	total: 3m 5s	remaining: 12m 19s
2000:	test: 0.7593266	best: 0.7593266 (2000)	total: 5m 59s	remaining: 8m 58s
3000:	test: 0.7619958	best: 0.7619958 (3000)	total: 8m 54s	remaining: 5m 56s
4000:	test: 0.7629983	best: 0.7630003 (3993)	total: 11m 36s	remaining: 2m 53s
4999:	test: 0.7632692	best: 0.7632692 (4999)	total: 14m 6s	remaining: 0us
bestTest = 0.7632692158
bestIteration = 4999
ROC AUC SCORE: 0.763269259841848
Fold 4 started


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6784991	best: 0.6784991 (0)	total: 222ms	remaining: 18m 29s
1000:	test: 0.7544881	best: 0.7544881 (1000)	total: 3m	remaining: 12m
2000:	test: 0.7594315	best: 0.7594315 (1999)	total: 5m 35s	remaining: 8m 22s
3000:	test: 0.7622085	best: 0.7622085 (2995)	total: 8m 10s	remaining: 5m 26s
4000:	test: 0.7641071	best: 0.7641071 (4000)	total: 10m 56s	remaining: 2m 43s
4999:	test: 0.7650754	best: 0.7650754 (4997)	total: 13m 36s	remaining: 0us
bestTest = 0.7650754452
bestIteration = 4997
Shrink model to first 4998 iterations.
ROC AUC SCORE: 0.7650754257275283
Fold 5 started


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6829214	best: 0.6829214 (0)	total: 224ms	remaining: 18m 37s
1000:	test: 0.7590761	best: 0.7590761 (1000)	total: 3m 10s	remaining: 12m 39s
2000:	test: 0.7646350	best: 0.7646350 (2000)	total: 6m 8s	remaining: 9m 11s
3000:	test: 0.7671057	best: 0.7671057 (3000)	total: 9m 2s	remaining: 6m 1s
4000:	test: 0.7679885	best: 0.7679913 (3988)	total: 11m 52s	remaining: 2m 57s
bestTest = 0.7680775821
bestIteration = 4177
Shrink model to first 4178 iterations.
ROC AUC SCORE: 0.7680775769335368


In [14]:
agg_model_scores.append(['CatBoostClassifier', metrics_catboost])

In [15]:
gc.collect()

0

In [16]:
best_cat = CatBoostClassifier(**best_params_catboost).fit(X_train, y_train)
print('ROC_AUC_SCORE:', roc_auc_score(y_test, best_cat.predict_proba(X_test)[:, 1]))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 235ms	remaining: 19m 35s
1000:	total: 3m 11s	remaining: 12m 43s
2000:	total: 6m 4s	remaining: 9m 6s
3000:	total: 8m 51s	remaining: 5m 54s
4000:	total: 11m 34s	remaining: 2m 53s
4999:	total: 14m 22s	remaining: 0us
ROC_AUC_SCORE: 0.7680675250876807


In [17]:
path_to_save = 'pickle_model'
model_filename_cat = f'{path_to_save}/CatClassifier_cr_sc.pkl'
with open(model_filename_cat, 'wb') as file:
    dill.dump(best_cat, file)

#### XGBClassifier

In [18]:
def GridSearchCV_XGB_early_stoppping(param_grid, fit_params, scorer, cv, X, y):
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()
    if isinstance(y, pd.Series):
        y = y.to_numpy()
        
    param_names, param_values = zip(*list(param_grid.items()))

    cv_best_iterations = defaultdict(list)
    cv_results = defaultdict(list)

    for train_index, test_index in cv.split(X, y):
        X_in, X_out = X[train_index], X[test_index]
        y_in, y_out = y[train_index], y[test_index]

        fit_params_cv = deepcopy(fit_params)
        fit_params_cv['eval_set'] = [(X_out, y_out)]

        for value_combination in product(*param_values):
            param_grid_cv = tuple(zip(param_names, value_combination))
            xgboost = XGBClassifier(**dict(param_grid_cv))

            xgboost.fit(X_in, y_in, **fit_params_cv)
            best_iteration = xgboost.get_num_boosting_rounds() if 'early_stopping_rounds' not in fit_params_cv else xgboost.best_iteration
            cv_best_iterations[param_grid_cv].append(best_iteration)

            score = scorer(xgboost, X_out, y_out)
            cv_results[param_grid_cv].append(score)
        
    best_params_xgb, score_list = max(cv_results.items(), key=lambda x: np.array(x[1]).mean())

    # Note that our XGBoost model may stop early,
    # so we calculate the mean of the actual number of estimators in each fold,
    # in place of the originally planned n_estimators after finishing cross validation.
    n_estimators = int(round(np.array(cv_best_iterations[best_params_xgb]).mean()))
    
    best_params_xgb = dict(best_params_xgb)
    best_params_xgb['n_estimators'] = n_estimators

    print ("Best score: {:.3f}".format(np.array(score_list).mean()))
    print ("Best Parameters: {}".format(best_params_xgb))
    
    return best_params_xgb

In [19]:
gc.collect()

0

In [20]:
param_grid = {
    'objective': ['binary:logistic'],
    'n_estimators': [1500],
    'max_depth': [4, 5, 6],
    'learning_rate': [0.1],
    'random_state': [42],
    'subsample': [0.8],
    'tree_method': ['gpu_hist'],
    'colsample_bytree': [0.8],
    'scale_pos_weight': [1],
    'eval_metric': ['auc'],
    'reg_lambda': [0.1]
}

fit_params = {
    'early_stopping_rounds': 100,
    'verbose': 300
}

scorer = make_scorer(roc_auc_score, greater_is_better=False)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_params_xgb = GridSearchCV_XGB_early_stoppping(param_grid, fit_params, scorer, kf, X_train, y_train)

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.66242
[300]	validation_0-auc:0.76333
[600]	validation_0-auc:0.76638
[900]	validation_0-auc:0.76703
[1200]	validation_0-auc:0.76720
[1369]	validation_0-auc:0.76720


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.67617
[300]	validation_0-auc:0.76469
[600]	validation_0-auc:0.76647
[733]	validation_0-auc:0.76665


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

[0]	validation_0-auc:0.66157
[300]	validation_0-auc:0.76476
[600]	validation_0-auc:0.76687
[900]	validation_0-auc:0.76773
[1122]	validation_0-auc:0.76782


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.68150
[300]	validation_0-auc:0.76583
[600]	validation_0-auc:0.76691
[608]	validation_0-auc:0.76689


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.69207
[300]	validation_0-auc:0.76666
[569]	validation_0-auc:0.76649
Best score: -0.501
Best Parameters: {'objective': 'binary:logistic', 'n_estimators': 1082, 'max_depth': 4, 'learning_rate': 0.1, 'random_state': 42, 'subsample': 0.8, 'tree_method': 'gpu_hist', 'colsample_bytree': 0.8, 'scale_pos_weight': 1, 'eval_metric': 'auc', 'reg_lambda': 0.1}


In [21]:
gc.collect()

304

In [22]:
models_xgboost = []
metrics_xgboost = []

cv_xgboost = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for i, (train_indexes, test_indexes) in enumerate(cv_xgboost.split(X_train, y_train), start=1):
    
    print(f'Fold {i} started:')
    
    model_xg = XGBClassifier(**best_params_xgb)
    
    y_tr, y_v = y_train.iloc[train_indexes], y_train.iloc[test_indexes]
    X_tr, X_v = X_train.iloc[train_indexes], X_train.iloc[test_indexes]
    
    fit_model_xg = model_xg.fit(X_tr, y_tr, eval_set=[(X_v, y_v)],
                                verbose=300, early_stopping_rounds=100)
    
    models_xgboost.append(fit_model_xg)
    metrics_xgboost.append(roc_auc_score(y_v, fit_model_xg.predict_proba(X_v)[:, 1]))
    
    print(f'Fold {i} training Done!')

Fold 1 started:


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.66118
[300]	validation_0-auc:0.76623
[600]	validation_0-auc:0.76884
[900]	validation_0-auc:0.76934
[1081]	validation_0-auc:0.76952
Fold 1 training Done!
Fold 2 started:


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.67243
[300]	validation_0-auc:0.76417
[600]	validation_0-auc:0.76668
[900]	validation_0-auc:0.76734
[1081]	validation_0-auc:0.76763
Fold 2 training Done!
Fold 3 started:


...

[0]	validation_0-auc:0.66967
[300]	validation_0-auc:0.76187
[600]	validation_0-auc:0.76416
[900]	validation_0-auc:0.76521
[1081]	validation_0-auc:0.76568
Fold 3 training Done!
Fold 4 started:


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.66127
[300]	validation_0-auc:0.76488
[600]	validation_0-auc:0.76737
[900]	validation_0-auc:0.76809
[1081]	validation_0-auc:0.76841
Fold 4 training Done!
Fold 5 started:


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.68075
[300]	validation_0-auc:0.76725
[600]	validation_0-auc:0.76970
[900]	validation_0-auc:0.77013
[1077]	validation_0-auc:0.77012
Fold 5 training Done!


In [23]:
agg_model_scores.append(['XGBClassifier', metrics_xgboost])

In [24]:
final_model_XGBoost = XGBClassifier(**best_params_xgb).fit(X_train, y_train, verbose=100)
print('ROC_AUC_SCORE:', roc_auc_score(y_test, final_model_XGBoost.predict_proba(X_test)[:, 1]))

ROC_AUC_SCORE: 0.7717257028672684


In [25]:
path_to_save = 'pickle_model'
model_filename = f'{path_to_save}/XGBClassifier_cr_sc.pkl'

with open(model_filename, 'wb') as file:
    dill.dump(final_model_XGBoost, file)

In [26]:
gc.collect()

483

#### LightGBM

In [27]:
def objective(trial):
    param_grid = {
        "n_estimators": trial.suggest_categorical("n_estimators", [2500]),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.1]),
        "max_depth": trial.suggest_int("max_depth", 3, 7),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.1, 1),
        "num_leaves": trial.suggest_int("num_leaves", 10, 100),
        "class_weight": trial.suggest_categorical("class_weight", ['balanced'])}  # to be filled in later
    
    classifier = lgbm.LGBMClassifier(**param_grid, metric='auc')
    classifier.fit(X_train, y_train)
    y_pred_proba = classifier.predict_proba(X_test)[:, 1]
    
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    return roc_auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials = 15)

lgbm_best_params = study.best_params
lgbm_best_score = study.best_value

print(lgbm_best_params)
print(lgbm_best_score)

[I 2023-08-14 06:39:05,468] A new study created in memory with name: no-name-9db835a0-a1d7-405c-8b3d-5fc2bde223b1


[LightGBM] [Warning] lambda_l2 is set=0.37456560679919826, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.37456560679919826


[I 2023-08-14 06:46:40,994] Trial 0 finished with value: 0.7704728281294952 and parameters: {'n_estimators': 2500, 'learning_rate': 0.1, 'max_depth': 3, 'lambda_l2': 0.37456560679919826, 'num_leaves': 10, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7704728281294952.


[LightGBM] [Warning] lambda_l2 is set=0.28147179727369687, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.28147179727369687


[I 2023-08-14 06:55:18,035] Trial 1 finished with value: 0.7531513953492175 and parameters: {'n_estimators': 2500, 'learning_rate': 0.1, 'max_depth': 6, 'lambda_l2': 0.28147179727369687, 'num_leaves': 38, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7704728281294952.


...

[LightGBM] [Warning] lambda_l2 is set=0.12534480958584876, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.12534480958584876


[I 2023-08-14 08:23:26,220] Trial 13 finished with value: 0.7671721590735916 and parameters: {'n_estimators': 2500, 'learning_rate': 0.1, 'max_depth': 4, 'lambda_l2': 0.12534480958584876, 'num_leaves': 67, 'class_weight': 'balanced'}. Best is trial 5 with value: 0.7705734558999735.


[LightGBM] [Warning] lambda_l2 is set=0.4173936510710393, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.4173936510710393


[I 2023-08-14 08:31:31,114] Trial 14 finished with value: 0.755969024791082 and parameters: {'n_estimators': 2500, 'learning_rate': 0.1, 'max_depth': 6, 'lambda_l2': 0.4173936510710393, 'num_leaves': 31, 'class_weight': 'balanced'}. Best is trial 5 with value: 0.7705734558999735.


{'n_estimators': 2500, 'learning_rate': 0.1, 'max_depth': 3, 'lambda_l2': 0.26973575050824705, 'num_leaves': 42, 'class_weight': 'balanced'}
0.7705734558999735


In [28]:
gc.collect()

45

In [29]:
cv_lgb = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

metrics_lgb = []
models_lgb = []


for i, (train_indexes, test_indexes) in enumerate(cv_lgb.split(X_train, y_train), start=1):
    
    print(f'kf {i} started!')
    
    X_tr, X_v = X_train.iloc[train_indexes], X_train.iloc[test_indexes]
    y_tr, y_v = y_train.iloc[train_indexes], y_train.iloc[test_indexes]
    
    LGBM = lgbm.LGBMClassifier(**lgbm_best_params, metric='auc').fit(X_train,  y_train)
    
    fit_model_LGBM = LGBM.fit(X_tr, y_tr, eval_set=[(X_v, y_v)],
                              verbose=300, early_stopping_rounds=100)
    
    models_lgb.append(fit_model_LGBM)
    metrics_lgb.append(roc_auc_score(y_v, fit_model_LGBM.predict_proba(X_v)[:, 1]))

kf 1 started!
[LightGBM] [Warning] lambda_l2 is set=0.26973575050824705, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.26973575050824705


C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[300]	valid_0's auc: 0.762362
[600]	valid_0's auc: 0.765871
[900]	valid_0's auc: 0.767119
[1200]	valid_0's auc: 0.767632
kf 2 started!
[LightGBM] [Warning] lambda_l2 is set=0.26973575050824705, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.26973575050824705


C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[300]	valid_0's auc: 0.761417
[600]	valid_0's auc: 0.76465
[900]	valid_0's auc: 0.765834
[1200]	valid_0's auc: 0.766267
kf 3 started!
[LightGBM] [Warning] lambda_l2 is set=0.26973575050824705, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.26973575050824705


...

[300]	valid_0's auc: 0.759724
[600]	valid_0's auc: 0.762591
[900]	valid_0's auc: 0.763613
[1200]	valid_0's auc: 0.764
kf 4 started!
[LightGBM] [Warning] lambda_l2 is set=0.26973575050824705, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.26973575050824705


C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[300]	valid_0's auc: 0.761757
[600]	valid_0's auc: 0.765246
[900]	valid_0's auc: 0.766378
[1200]	valid_0's auc: 0.766984
kf 5 started!
[LightGBM] [Warning] lambda_l2 is set=0.26973575050824705, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.26973575050824705


C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[300]	valid_0's auc: 0.764522
[600]	valid_0's auc: 0.76736
[900]	valid_0's auc: 0.768501
[1200]	valid_0's auc: 0.768947


In [30]:
gc.collect()

494

In [31]:
LGBMClassifier = lgbm.LGBMClassifier(**lgbm_best_params, metric='auc').fit(X_train, y_train)

print('ROC_AUC_SCORE:', roc_auc_score(y_test, LGBMClassifier.predict_proba(X_test)[:, 1]))

[LightGBM] [Warning] lambda_l2 is set=0.26973575050824705, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.26973575050824705
ROC_AUC_SCORE: 0.7705734558999735


In [32]:
agg_model_scores.append(['LGBMClassifier', metrics_lgb])

In [33]:
path_to_save = 'pickle_model'
model_filename_lgbm = f'{path_to_save}/LGBMClassifier_cr_sc.pkl'
with open(model_filename_lgbm, 'wb') as file:
    dill.dump(LGBMClassifier, file)

In [34]:
gc.collect()

45

In [35]:
res = pd.concat([make_dataset(agg_model_scores[0]), make_dataset(agg_model_scores[1])], axis=0)
res = pd.concat([res, make_dataset(agg_model_scores[2])], axis=0).reset_index(drop=True)

[['CatBoostClassifier', [0.7670122242726762, 0.7643396464970058, 0.763269259841848, 0.7650754257275283, 0.7680775769335368]]]
[['XGBClassifier', [0.7695185696360035, 0.7676779642618016, 0.7656862374419516, 0.7684325696551452, 0.7702363051127683]]]
[['LGBMClassifier', [0.7677977698966809, 0.7664040654815093, 0.7642131478118648, 0.7670707869336656, 0.7690106656785134]]]


In [36]:
res.to_csv('results_cv.xlsx')

In [37]:
res

,Модель,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean roc_auc_score
0,CatBoostClassifier,0.767012,0.764340,0.763269,0.765075,0.768078,0.765555
1,XGBClassifier,0.769519,0.767678,0.765686,0.768433,0.770236,0.768310
2,LGBMClassifier,0.767798,0.766404,0.764213,0.767071,0.769011,0.766899
